In [3]:
import tushare as ts
import numpy as np
ts.set_token("8c6e5026fd3e2b5f96f15553d72f84eaaea8182a9157b783a41b1be0")
pro = ts.pro_api()

In [4]:
all_codes = list(set(pro.cb_basic(fields="ts_code").ts_code))
all_codes

['125572.SZ',
 '110803.SH',
 '123079.SZ',
 '123100.SZ',
 '123085.SZ',
 '123049.SZ',
 '125717.SZ',
 '110010.SH',
 '113594.SH',
 '128064.SZ',
 '110232.SH',
 '100009.SH',
 '128098.SZ',
 '128094.SZ',
 '128138.SZ',
 '124005.SZ',
 '110049.SH',
 '123103.SZ',
 '110971.SH',
 '128068.SZ',
 '123012.SZ',
 '128099.SZ',
 '110064.SH',
 '110043.SH',
 '113030.SH',
 '123065.SZ',
 '128049.SZ',
 '128091.SZ',
 '128058.SZ',
 '123025.SZ',
 '113040.SH',
 '113591.SH',
 '113619.SH',
 '113580.SH',
 '127019.SZ',
 '100220.SH',
 '110039.SH',
 '128055.SZ',
 '125932.SZ',
 '128048.SZ',
 '128011.SZ',
 '128129.SZ',
 '113050.SH',
 '113523.SH',
 '110802.SH',
 '123073.SZ',
 '123029.SZ',
 '123051.SZ',
 '113566.SH',
 '123002.SZ',
 '113034.SH',
 '128007.SZ',
 '123033.SZ',
 '110807.SH',
 '128095.SZ',
 '127031.SZ',
 '128047.SZ',
 '113547.SH',
 '121003.SZ',
 '125630.SZ',
 '128063.SZ',
 '110874.SH',
 '128140.SZ',
 '110060.SH',
 '110028.SH',
 '110080.SH',
 '113007.SH',
 '123045.SZ',
 '127023.SZ',
 '100096.SH',
 '110598.SH',
 '1100

### 记录每只股票转股价变化表

In [74]:
# import os
# from tqdm import tqdm
# # all_codes = get_all_codes()
# os.chdir(r'E:\quantTradeLeanner\QuantTrader\tushare\convert_table')
# os.getcwd()
# import time # 每分钟最多访问200次
# for code in tqdm(all_codes):
#     df = pro.cb_price_chg(ts_code=code,fields="change_date,convert_price_initial,convertprice_aft")
#     df.to_csv(code.split('.')[0]+'_'+code.split('.')[1]+'.csv')
#     time.sleep(0.1)
#     del df

100%|████████████████████████████████████████████████████████████████████████████████| 648/648 [04:32<00:00,  2.38it/s]


In [6]:
### 加载所有转债的代码
# def get_all_codes():
#     '''
#         return: 加载所有转债的代码
#     '''
#     os.chdir(r'E:\quantTradeLeanner\QuantTrader\tushare')
#     all_codes = np.load('all_codes_20180101_20210601.npy')
#     return all_codes

### 获取债股票代码转换字典
def get_d2s_dict(all_codes):
    '''
        all_codes: 这两年所有的债券代码
        return: 这两年所有的债券转股票代码
    '''
    df = pro.cb_basic(fields="ts_code,stk_code")
#     df = df[df.ts_code.isin(all_codes)]
    df.reset_index()
    det2share_dict = df.set_index('ts_code').T.to_dict('list')
    return det2share_dict

### 查询债券价格
def get_daily_table(time):
    '''
        time: 时间
        return : 两列df, 债券代码和债券价格
    '''
    daily_table = pro.cb_daily(trade_date=time,fields='ts_code,close')
    daily_table.columns = ['ts_code','det_price']
    return daily_table

# 查询股票价格
def get_share_price(codes,time,det2share_dict):
    '''
        code: 当日代码列表
        time: 检索时间
        det2share_dict: 债券对应的正股代码字典
        return : pd.DataFrame 当日的正股价格序列
        作用: 检索当日可转债对应的正股价
    '''
    share_price = []
    for code in codes:
        share_code = det2share_dict[code][0]
        tt = np.array(pro.daily(ts_code=share_code, start_date=time, end_date=time).close)
        if len(tt) == 0:
            share_price.append(0)
        else:
            share_price.append(tt[0])
            
#         share_price.append(np.array(pro.daily(ts_code=share_code, start_date=time, end_date=time).pre_close))
#         print(np.array(pro.daily(ts_code=share_code, start_date=time, end_date=time).pre_close))
#         print(pro.daily(ts_code=share_code, start_date=time, end_date=time).pre_close)
#     break
    share_price = pd.DataFrame(share_price,columns=['share_price'])
    return  share_price

# 查询转股价格
def get_convprice_per_det(codes,time):
    '''
        code: 当日代码列表
        time: 检索时间
        return : pd.DataFrame 当日的转股价序列
        作用: 检索当日可转债对应的转股价
    '''
    os.chdir(r'E:\quantTradeLeanner\QuantTrader\tushare\convert_table')
    conv_price_list = []
    for code in codes:
        code_info = pd.read_csv(code.split('.')[0]+'_'+code.split('.')[1]+'.csv')
        date_list = list(code_info.change_date)
        price_list = list(code_info.convertprice_aft)
        price_list[0]=code_info.convert_price_initial[0]
        flag = 0
        i = 0
        for d in date_list:
            i = i + 1
            if int(time) > int(d):
                flag = i
            else:
                break
        #     if flag == 0 or flag ==1:
        conv_price = price_list[flag-1]
        conv_price_list.append(conv_price)
    conv_price_list = pd.DataFrame(conv_price_list,columns=['conv_price'])
    return conv_price_list
def get_final_table(daily_table,share_price,conv_price):
    '''
        return: 最终表格,包括溢价率
    '''
    df = pd.concat([daily_table,share_price,conv_price],axis=1)
    mid_value = (100 / df.conv_price) * df.share_price
    over_rate = (df.det_price-mid_value)/mid_value
    over_rate = pd.DataFrame(over_rate,columns=['over_rate'])
    df1 = pd.concat([df,over_rate],axis=1)
    df1 = df1.sort_values(by='over_rate',ascending=False).reset_index()
    return df1

def save_table(start_time,end_time):
    
    for time in tqdm(range(start_time,end_time)):
     
        is_empty = True if len(pro.cb_daily(trade_date=str(time),fields='ts_code')) == 0 else False 
        if is_empty:
            pass
        else:
            daily_table = get_daily_table(time)
            share_price = get_share_price(daily_table.ts_code,time,det2share_dict)
            conv_price = get_convprice_per_det(daily_table.ts_code,time)
            df = get_final_table(daily_table,share_price,conv_price)
            os.chdir(r'E:\quantTradeLeanner\QuantTrader\tushare\daily_table')
            df.to_csv(str(time)+'.csv')
            print(str(time)+'.csv saved!')

In [140]:
# start_time,end_time = 20181228,20181231
# save_table(start_time,end_time)

In [141]:
# start_time,end_time = 20190329,20191231
# save_table(start_time,end_time)

In [142]:
# start_time,end_time = 20200101,20201231
# save_table(start_time,end_time)

In [143]:
# start_time,end_time = 20210101,20210613
# save_table(start_time,end_time)

### 获取20180404当天的溢价率表

In [144]:
import numpy as np 
import os 
from tqdm import tqdm
import pandas as pd


all_codes = get_all_codes()
det2share_dict = get_d2s_dict(all_codes)

time = '20180404'
daily_table = get_daily_table(time)
share_price = get_share_price(daily_table.ts_code,time,det2share_dict)
conv_price = get_convprice_per_det(daily_table.ts_code,time)
get_final_table(daily_table,share_price,conv_price)

,index,ts_code,det_price,share_price,conv_price,over_rate
0,26,113502.SH,98.710,0.00,45.48,inf
1,24,113016.SH,102.100,0.00,23.00,inf
2,52,123007.SZ,117.050,0.00,45.21,inf
3,12,128013.SZ,92.385,4.58,10.00,1.017140
4,9,113010.SH,105.300,4.86,9.30,1.015000
...,...,...,...,...,...,...
57,49,110042.SH,114.690,15.76,14.29,0.039924
58,61,128036.SZ,100.032,9.35,9.72,0.039905
59,56,123008.SZ,135.200,65.28,47.01,-0.026386
60,28,110039.SH,139.440,26.61,18.36,-0.037911


### 数据填充和数据清洗

In [15]:
import os
from tqdm import tqdm
import pandas as pd
os.chdir(r'E:\quantTradeLeanner\QuantTrader\tushare\daily_table')
os.getcwd()

'E:\\quantTradeLeanner\\QuantTrader\\tushare\\daily_table'

In [11]:
file_list = os.listdir()
len(file_list)

834

In [16]:


lost_info = []
for file in tqdm(file_list):
#     file = '20180404.csv'
    df = pd.read_csv(file)
    df = df.replace([np.inf, -np.inf], np.nan)
    if df.over_rate.isna().sum()>0:
        lost_dict = {
            'file':file,
            'nan_nums':df.over_rate.isna().sum(),
            'nan_det_codes':list(df[df.over_rate.isna()].ts_code)
        }
        lost_info.append(lost_dict)
    else:
        pass

# break

100%|████████████████████████████████████████████████████████████████████████████████| 834/834 [00:15<00:00, 53.74it/s]


In [17]:
list(df[df.over_rate.isna()].ts_code)

[]

In [18]:
all_nums = 0
all_lost_codes = set()
for info in tqdm(lost_info):
#     print(info)
    all_nums += info['nan_nums']
    all_lost_codes.update(info['nan_det_codes'])
print(all_nums)

100%|████████████████████████████████████████████████████████████████████████████| 400/400 [00:00<00:00, 400602.10it/s]

572


In [19]:
lost_info

[{'file': '20180108.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180109.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180110.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180111.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180112.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180115.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180116.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180117.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180118.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180119.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180122.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180123.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180124.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180125.csv', 'nan_nums': 

In [20]:
len(all_lost_codes)

44

In [21]:
det2share_dict = get_d2s_dict(all_codes)

In [26]:
for code in list(all_lost_codes):
    print(code,det2share_dict[code][0])

110077.SH 600461.SH
113595.SH 603007.SH
113015.SH 601012.SH
128036.SZ 002548.SZ
113008.SH 601727.SH
123031.SZ 300655.SZ
110071.SH 600929.SH
123021.SZ 300168.SZ
113027.SH 601020.SH
123010.SZ 300422.SZ
110056.SH 600487.SH
123061.SZ 300424.SZ
123043.SZ 300645.SZ
110048.SH 600483.SH
113502.SH 603822.SH
128022.SZ 002707.SZ
113552.SH 603960.SH
127018.SZ 000761.SZ
123052.SZ 300665.SZ
123068.SZ 300657.SZ
113517.SH 603019.SH
123007.SZ 300409.SZ
113576.SH 603557.SH
123004.SZ 300197.SZ
113527.SH 603518.SH
110053.SH 600919.SH
113515.SH 603588.SH
127030.SZ 000301.SZ
113537.SH 603348.SH
123013.SZ 300539.SZ
113524.SH 603677.SH
110033.SH 600755.SH
128090.SZ 002510.SZ
127025.SZ 000401.SZ
128037.SZ 002542.SZ
128040.SZ 002758.SZ
128054.SZ 002891.SZ
113016.SH 601127.SH
128043.SZ 002793.SZ
110067.SH 600909.SH
128086.SZ 002074.SZ
128010.SZ 002245.SZ
128012.SZ 002496.SZ
123044.SZ 300427.SZ


In [29]:
pro.daily(ts_code='300409.SZ', start_date=20180101, end_date=20180601)

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,300409.SZ,20180601,46.00,47.88,45.80,46.79,46.10,0.69,1.50,33462.65,157044.633
1,300409.SZ,20180531,46.84,48.40,46.00,46.10,45.98,0.12,0.26,40857.90,191442.366
2,300409.SZ,20180530,47.26,48.50,45.51,45.98,48.90,-2.92,-5.97,37136.31,175843.248
3,300409.SZ,20180529,49.86,51.50,48.61,48.90,50.50,-1.60,-3.17,35965.70,179521.232
4,300409.SZ,20180528,50.23,51.37,47.00,50.50,51.74,-1.24,-2.40,78068.86,385298.502
5,300409.SZ,20180525,58.79,58.80,51.74,51.74,57.49,-5.75,-10.00,88423.83,473085.069
6,300409.SZ,20180524,57.39,60.25,57.00,57.49,57.53,-0.04,-0.07,55309.20,324572.701
7,300409.SZ,20180523,57.00,60.68,56.50,57.53,57.37,0.16,0.28,67981.17,400607.403
8,300409.SZ,20180522,54.20,57.87,54.01,57.37,54.50,2.87,5.27,56792.01,319204.678
9,300409.SZ,20180521,53.65,55.76,53.65,54.50,53.43,1.07,2.00,48414.24,264810.272


In [105]:
for file in tqdm(file_list):
#     file = '20180404.csv'
    df = pd.read_csv(file)
# file = '20180409.csv'
#     df = pd.read_csv(file)
#     df = df.replace([np.inf, -np.inf], np.nan)
#     df = df.dropna().reset_index(drop=True)
    df = df.drop(['Unnamed: 0'],axis=1)
    df.to_csv(str(file))
#     df

100%|████████████████████████████████████████████████████████████████████████████████| 834/834 [00:15<00:00, 53.98it/s]


In [106]:
df

,ts_code,det_price,share_price,conv_price,over_rate
0,123013.SZ,357.40,7.55,9.13,3.321936
1,128041.SZ,280.71,5.99,6.82,2.196064
2,128062.SZ,79.76,4.29,16.25,2.021212
3,123015.SZ,230.00,4.54,5.79,1.933260
4,123072.SZ,128.18,25.73,56.12,1.795749
...,...,...,...,...,...
361,113534.SH,138.79,21.40,15.18,-0.015499
362,128032.SZ,134.49,13.52,9.87,-0.018183
363,123089.SZ,125.56,10.26,8.00,-0.020975
364,113509.SH,221.48,34.50,15.25,-0.020994


In [82]:
str(file)

'20210611.csv'

In [110]:
file = '20180810.csv'
df = pd.read_csv(file)
# df = df.drop(['Unnamed: 0'],axis=1)
# df = df.drop(['Unnamed: 0'],axis=1)
# df.to_csv(str(file))
df

,Unnamed: 0,ts_code,det_price,share_price,conv_price,over_rate
0,0,128013.SZ,87.221,3.60,9.98,1.417960
1,1,128012.SZ,74.985,2.55,7.71,1.267194
2,2,113008.SH,102.130,5.35,10.37,0.979604
3,3,128021.SZ,92.739,5.30,11.23,0.965017
4,4,123003.SZ,95.225,12.08,24.18,0.906077
...,...,...,...,...,...,...
73,73,128025.SZ,102.363,15.89,16.10,0.037158
74,74,123008.SZ,138.500,41.88,31.23,0.032797
75,75,123009.SZ,129.500,35.37,27.69,0.013813
76,76,110032.SH,129.990,9.53,7.41,0.010730


In [ ]:
for file in tqdm(file_list):
    df = pd.read_csv(file)
    df = df.drop(['Unnamed: 0'],axis=1)
    df.to_csv(str(file))

 32%|█████████████████████████▊                                                      | 269/834 [00:03<00:07, 79.61it/s]